In [3]:
# Import needed packages
import nibabel as nib
import numpy as np
from bids import BIDSLayout
from nipype.interfaces.ants import ApplyTransforms
from nipype.interfaces import fsl
# Define useful functions 
def get_FLAIR(subj):
    '''This function returns the name of the FLAIR file.'''
    FLAIR_filenames = layout.get(subject = subj, datatype = 'anat', desc = 'preproc', label = 'FLAIR', extension = 'nii.gz', return_type='file')
    FLAIR = FLAIR_filenames[0]
    return FLAIR

def get_transformation(subj):
    '''Returns the transformation matrix filename from the subject 'subj'. Note that ANTS want the inverse transformation, so if we want to go
    from subject space to MNI, we actually have to pass to ANTS the MNI-to-T1 transformation matrix'''
    # See https://neurostars.org/t/extracting-individual-transforms-from-composite-h5-files-fmriprep/2215    
    transformation = [f for f in layout.get(subject = subj, datatype = 'anat', extension = 'h5', return_type='file') if "from-MNI152NLin2009cAsym_to-T1w" in f]
    transform = transformation[0]
    return transform

def get_T1(subj):
    '''Returns the T1 file for the subject 'subj' in subject space.'''
    all_t1_filenames = layout.get(subject = subj, datatype = 'anat', desc = 'preproc', extension = 'nii.gz', return_type='file')
    mni2009_filenames = layout.get(subject = subj, datatype = 'anat', desc = 'preproc', space = 'MNI152NLin2009cAsym', extension = 'nii.gz', return_type='file')
    mni6_filenames = layout.get(subject = subj, datatype = 'anat', desc = 'preproc', space = 'MNI152NLin6Asym', extension = 'nii.gz', return_type='file')
    t1_filenames = [f for f in all_t1_filenames if f not in mni2009_filenames + mni6_filenames if 'FLAIR' not in f]
    t1_filename = t1_filenames[0]
    return t1_filename

def get_T1_mask(subj):
    '''Returns the T1 file-mask for the subject 'subj' in subject space. '''
    t1_mask_file = layout.get(subject = subj, extension = 'nii.gz', space = 'MNI152NLin2009cAsym', return_type='file', desc = 'brain', suffix = 'mask')
    t1_mask = t1_mask_file[0]
    return t1_mask

def get_T1_MNI(subj):
    '''Returns the T1 file for the subject 'subj' in MNI2009 space'''
    mni2009_filenames = layout.get(subject = subj, datatype = 'anat', desc = 'preproc', space = 'MNI152NLin2009cAsym', extension = 'nii.gz', return_type='file')
    mni2009_filename = mni2009_filenames[0]
    return mni2009_filename

def preprocess_FLAIR(subj, flair_name, t1_name, t1_mask_name):
    '''This function first reorient the FLAIR image for subject 'subj' in MNI space (note, it is just rotation/translation, not registration)
    Then it performs brain extraction (BET) for the FLAIR. Then it performs BET on the T1 by simply applying the related brain-mask, because
    BET was already perfomed as part of fmriprep. BET images are needed for registration. It then registers the FLAIR-BET to T1-BET in subject 
    space. It saves all output in a temp. directory. Returns the filename of the registered FLAIR.'''
    
    # Set the filenames
    reoriented_FLAIR_file = f'{temp_dir}/sub-{subj}/sub-{subj}_FLAIR_reoriented_to_std.nii.gz'
    reoriented_FLAIR_BET_file = f'{temp_dir}/sub-{subj}/sub-{subj}_FLAIR_reoriented_to_std_BET.nii.gz'
    masked_t1_file = f'{temp_dir}/{subj}_masked_t1_for_registration.nii.gz'
    flair_registered_to_t1_name = f'{BIDS_dir}/sub-{subj}/anat/sub-{subj}_space-MNI152NLin2009cAsym_desc-preproc_FLAIR-registeredToT1.nii.gz'
    # Reorient to MNI standard space (only rotation/translation, no registration)
    reorient = fsl.Reorient2Std()
    reorient.inputs.in_file = flair_name
    reorient.inputs.out_file = reoriented_FLAIR_file
    reorient.cmdline
    reorient.run() 
    # Perform BET on FLAIR
    btr = fsl.BET()
    btr.inputs.in_file = reoriented_FLAIR_file
    btr.inputs.out_file = reoriented_FLAIR_BET_file
    btr.cmdline
    btr.run() 
    #Load the T1 and T1 mask in subject space
    t1 = nib.load(t1_name)
    t1_mask = nib.load(t1_mask_name)
    # Create a masked T1 (= as BET T1, since the mask was obtained from the BET image in fmriprep)
    masked_t1 = nimg.math_img('a * b', a = t1, b = t1_mask)
    # Save the new masked T1 file
    nib.save(masked_t1, masked_t1_file)
    # Register the FLAIR to the T1 in subject space 
    flt = fsl.FLIRT(cost_func='mutualinfo')
    flt.inputs.in_file = reoriented_FLAIR_BET_file
    flt.inputs.reference = masked_t1_file
    flt.inputs.output_type = 'NIFTI_GZ'
    flt.inputs.out_file = flair_registered_to_t1_name
    flt.cmdline 
    flt.run() 
    return flair_registered_to_t1_name

def transform_registered_FLAIR_to_MNI(subj, registered_flair, t1_mni_name, transform):
    '''This function applies the transformation calculated by fmriprep and stored as .h5 file to the registered FLAIR and it saves
    the file with the FLAIR in MNI space registered to the T1'''
    # Set the BIDS compliant name for the flair
    flair_MNI_name = f'{BIDS_dir}/sub-{subj}/anat/sub-{subj}_space-MNI152NLin2009cAsym_desc-preproc_FLAIR.nii.gz'
    # Apply the transformation. Note that ANTS works with the opposite transformation.
    at = ApplyTransforms()
    at.inputs.input_image = registered_flair
    at.inputs.reference_image = t1_mni_name
    at.inputs.output_image = flair_MNI_name
    at.inputs.transforms = transform
    at.inputs.dimension = 3
    at.cmdline
    at.run()

ValueError: BIDS root does not exist: /home/riccardo/ADNI_Hopf/Data/ALL

In [ ]:
# change to your BIDS directory 
BIDS_dir = '/home/riccardo/ADNI_Hopf/Data/ALL'
# set a temporary directory
temp_dir = '/home/riccardo/ADNI_Hopf/Temp'
# Get the layout of the bids folder
layout = BIDSLayout(BIDS_dir, validate = False, config = ['bids', 'derivatives'])
# Get the list of subjects
subjs = layout.get_subjects()
s = len(subjs)
for c, subj in enumerate(subjs):
    t1_name = get_T1(subj)
    flair_name = get_FLAIR(subj)
    transform_name = get_transformation(subj)
    t1_mask_name = get_T1_mask(subj)
    t1_mni_name = get_T1_MNI(subj)
    print(f'Preprocessing and registering FLAIR to T1 for sub-{subj}... ({c+1}/{s})')
    flair_registered_to_t1_name = preprocess_FLAIR(subj, flair_name, t1_name, t1_mask_name)
    print(f'Transforming FLAIR to MNI space for sub-{subj}... ({c+1}/{s})')
    transform_registered_FLAIR_to_MNI(subj, flair_registered_to_t1_name, t1_mni_name, transform_name)
print('Done')